In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
# dbutils.widgets.text("proc_date", "")
# proc_date = dbutils.widgets.get("proc_date")

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_rop',
    'problem_article',
    proc_date,
    'temp_rop_problem_article'
)

In [0]:
bronze_count = spark.sql("SELECT count(1) FROM temp_rop_problem_article").collect()[0][0]

In [0]:
if bronze_count > 0:
    spark.sql(f"""CREATE OR REPLACE TABLE udp_wcm_dev.udp_wcm_silver_rop.problem_article
    TBLPROPERTIES (
    'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
    'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
    ) AS
    WITH
    TEMP AS (
        SELECT
            *,
            TO_DATE(TO_TIMESTAMP(updated_at, 'M/d/yyyy h:m:s a')) AS updated_date
        FROM temp_rop_problem_article
    )
    SELECT
        CASE
            WHEN domain = 'Winmart' THEN '1500'
            WHEN domain = 'Winmart+' THEN '2000'
        END AS business_unit,
        * EXCEPT (index, domain, updated_at, updated_date),
        TO_TIMESTAMP(updated_at, 'M/d/yyyy h:m:s a') AS updated_at,
        FARM_FINGERPRINT(CONCAT(domain, '|', area, '|', sku, '|', updated_date)) AS hash_id
    FROM TEMP
    """)
    